In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [2]:
model_client = OpenAIChatCompletionClient(
    model = "gemini-1.5-flash-8b",
    api_key = api_key,
)

In [8]:
poet = AssistantAgent(
    name="poet",
    description="A poet who writes poems based on user prompts.",
    model_client=model_client,
    system_message="You are a Great poet Who is Inpired by william shakesphere. Write poems based on the user's prompts and after generation just say 'THE END'",
)

In [9]:
from autogen_agentchat.conditions import TextMentionTermination

termination_condition = TextMentionTermination('THE END')

In [10]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[poet],
    termination_condition=termination_condition,
)

In [11]:
from autogen_agentchat.messages import TextMessage
async def test_poet():
    task = TextMessage(content="Write a poem about the Moon And the Earth.", source='User')
    result = await team.run(task=task)

    for msg in result.messages:
        print(f"{msg.source}: {msg.content}")

await test_poet()

User: Write a poem about the Moon And the Earth.
poet: From realms of velvet, silver-bright, the Moon doth gaze,
Upon a sphere of emerald, bathed in sun's embrace.
A silent dance, through cosmic night, they gently weave,
Two worlds entwined, in shadows deep, their secrets to conceive.

The Earth, a jewel, with azure veins, and forests dark and grand,
With whispered sighs, and rising rains, a fertile, living hand.
Upon her bosom, life takes hold, in verdant, vibrant hues,
While Luna watches, old and cold, her timeless, silent muse.

Through star-strewn paths, their journey's spun, a celestial thread,
Attraction's pull, the tides are run, by forces yet unsaid.
The Moon, a mirror, reflects the sun, a ghostly, pale delight,
A whispered hymn, a silent gun, to endless, starlit night.

Her craters deep, a history untold, in lunar dust so fine,
While Earth, with tales of ages old, in every breeze does shine.
Two sisters, bound, in cosmic grace, eternally they roam,
A timeless dance, in time an

### Saving and Loading Agents

In [29]:
assistant = AssistantAgent(
    name="TouristAgent",
    description="A simple assistant agent.",
    model_client=model_client,
    system_message="You are Best Tourist Planner Agent Who's expertise in planning tours to worlds wonders.",
)


In [31]:
from autogen_core import CancellationToken
response = await assistant.on_messages(
    [TextMessage(content="Best plan to cover most wonders of egypt", source='User')],
    cancellation_token=CancellationToken()   
)

print(f"Assistant Response: {response.chat_message.content}")

Assistant Response: A plan to cover most wonders of Egypt in a way that is both efficient and enjoyable requires careful consideration of your interests and timeframe.  A "most wonders" approach can be overwhelming if done in a rush.  Here are a few options depending on your preferences:

**Option 1:  The Classic 10-14 Day Egyptian Wonders Tour (Focus on Iconic Sites)**

* **Duration:** 10-14 days.  This allows for a good pace without feeling rushed.
* **Focus:**  Iconic sights like the Pyramids of Giza, Sphinx, Saqqara, Memphis, Karnak Temple, Luxor Temple, Valley of the Kings, and possibly a few select sites in the south like Abu Simbel.
* **Itinerary (Example, 10 days):**
    * **Days 1-3:** Cairo – Pyramids, Sphinx, Egyptian Museum, maybe a Sound and Light show.
    * **Days 4-5:**  Luxor – Karnak and Luxor Temples, Valley of the Kings, possible hot air balloon ride.
    * **Days 6-8:**  Aswan – Abu Simbel (day trip), Philae Temple, High Dam.
    * **Days 9-10:**  Possible day trip

In [32]:
agent_state = await assistant.save_state()

In [33]:
new_assistant = AssistantAgent(
    name="Assistant",
    description="A simple assistant agent.",
    model_client=model_client,
    system_message="You are a helpful assistant.",
)

In [34]:
await new_assistant.load_state(agent_state)

In [35]:
response = await new_assistant.on_messages(
    [TextMessage(content="can you tell what asked before?", source='User')],
    cancellation_token=CancellationToken()
)

print(response.chat_message.content)

You asked for the best plan to cover most of the wonders of Egypt.

